In [1]:
# altair to visualize the data. read in pickle
# determine df shape for this to work
# import birdseye and snoop
import snoop
import altair as alt
# from vega_datasets import data
from typing import Optional, Dict, List, Any, Union, Tuple
import pydantic
from pydantic import ValidationError, validator, root_validator, Field, constr, BaseModel
from pydantic_docx import Docx_Paragraph_and_Runs, read_docx, extract_features #type:ignore
from pydantic_docx_processor import create_sized_dataframe, expand_dataframe #type:ignore
import re
import json
from itertools import compress, chain
from datetime import datetime
import pandas as pd
import numpy as np
from functools import partial
import pickle

In [146]:
# def test_func(input_stuff):
#    # print(input_stuff)
#    def item(**kwargs):
#       print('test')
#       print(dir(input_stuff))
#       print(dict(input_stuff))
#       print(kwargs)
#       result = input_stuff.parse_obj(kwargs)
#       print(result)
#       return result
#       # print(kwargs)
#    return item


def chS_update(obj,attrs: Dict[str,Any],in_place = False):
   """check self update - calls setattr but passes in a dict.
   calls obj.dict and for each key:value pair in the attrs dict, passes a dict containing the key:value pair, 
   as well as "current_state":obj.dict(). This will allow pydantic models to check current states for values with complex conditional validations
   'attrs': acceps a dict of attributes names:values
   """
   if not in_place:
      obj = obj.copy()
   current_state = obj.dict()
   for key,val in attrs.items():
      print(key)
      print(val)
      obj = obj.validate(*{key:val,'current_state':current_state})
      setattr(obj,key,)
   if not in_place:
      return obj

# @test_func
class masked_strings(pydantic.BaseModel):
   checked : bool
   present : Optional[bool]
   strings : Optional[List[str]]
   masks : Optional[np.ndarray]
   #validate assignment, ensure lengths
   #functions for regex that update submasks
   #functions for regex that only check whole runs

   class Config:
      validate_all = True
      validate_assignment = True
      # smart_union = True 
      arbitrary_types_allowed = True 
      extra = 'forbid'


   @root_validator(pre=False)
   def validate_lengths(cls,values):
      enforce_lengths_on = ['strings','masks']
      # lists = []
      # for feat in enforce_lengths_on:
      #    if feat not in values:
      #       lists.append(current_state)
      print(cls)
      if 'strings' in values and 'masks' in values and len(values['masks']) != len(values['strings']):
         raise ValueError('strings and mask do not match')
      current_state = values.get('current_state',False)
      if current_state:
         if 'strings' in current_state and len(values['masks']) != len(current_state['strings']):
            raise ValueError('strings and mask do not match')
      return values

   def update(self,**kwargs):
      current_state = self.dict()
      current_state.update(kwargs)
      # kwargs['current_state'] = self.obj.dict()
      print(kwargs)
      print(self.parse_obj(current_state))
      # self = self.parse_obj(current_state)
      try:
         self.validate(current_state)
         for k,v in kwargs.items():
            setattr(self,k,v)
      except:
         raise
class check_wrapper(pydantic.BaseModel):
   obj : masked_strings

   def update(self,**kwargs):
      current_state = self.obj.dict()
      current_state.update(kwargs)
      # kwargs['current_state'] = self.obj.dict()
      print(kwargs)
      self.obj = self.obj.validate(current_state)
test = masked_strings(checked=True,present=True,strings=['i','love','stacy'],masks=np.array([True,True,True]))
# test = masked_strings.parse_obj({'checked': True, 'present': True, 'strings': ['i', 'love', 'stacy'], 'masks': np.array([ True,  True,  True])})
# print(test.dict())
# print(dir(test))
ctest = check_wrapper(obj=test)
# test.validate({ 'masks': np.array([ True, True, True])})
# test.validate({'checked': True, 'present': True, 'strings': ['i', 'love', 'stacy'], 'masks': np.array([ True,  True,  True])})
test.update(**{'masks': np.array([False,False,False])})
test
# print(test.strings +['plots'])
# chS_update(test,{'strings':test.strings +['plots']})
# test.strings
# test.dict('a')

<class '__main__.masked_strings'>
{'masks': array([False, False, False])}
<class '__main__.masked_strings'>
checked=True present=True strings=['i', 'love', 'stacy'] masks=array([False, False, False])
<class '__main__.masked_strings'>
<class '__main__.masked_strings'>


masked_strings(checked=True, present=True, strings=['i', 'love', 'stacy'], masks=array([False, False, False]))

test
['Config', '__abstractmethods__', '__annotations__', '__class__', '__class_vars__', '__config__', '__custom_root_type__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__exclude_fields__', '__fields__', '__fields_set__', '__format__', '__ge__', '__get_validators__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__include_fields__', '__init__', '__init_subclass__', '__iter__', '__json_encoder__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__post_root_validators__', '__pre_root_validators__', '__pretty__', '__private_attributes__', '__reduce__', '__reduce_ex__', '__repr__', '__repr_args__', '__repr_name__', '__repr_str__', '__schema_cache__', '__setattr__', '__setstate__', '__signature__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__try_update_forward_refs__', '__validators__', '_abc_impl', '_calculate_keys', '_copy_and_set_values', '_decompose_class', '_enforce_dict_if_root', '_get_value', '_init_private_attributes', '_iter',

AttributeError: 'NoneType' object has no attribute 'strings'